In [ ]:
!pip install networkx
!pip install nltk

In [ ]:
import networkx as nx
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
sentences = [
    "I have symptoms. How can I get releived of these?",
    "Am I the suspect?",
    "Should I take it",
    "Who manufactures cetirizine?",
    "Who manufactures bromocriptine?",
    "Who makes nulytely, and where can I buy it?"
]

In [ ]:
def preprocess_text(text):
    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()

    sentences = sent_tokenize(text)
    words_in_sentences = [word_tokenize(sentence) for sentence in sentences]
    words_in_sentences = [[word.lower() for word in words if word.isalnum()] for words in words_in_sentences]
    words_in_sentences = [[stemmer.stem(word) for word in words if word not in stop_words] for words in words_in_sentences]

    return sentences

In [ ]:
# Function to create a graph based on sentences
def create_graph(sentences):
    graph = nx.Graph()

    for sentence in sentences:
        for word in sentence:
            if not graph.has_node(word):
                graph.add_node(word)

    for sentence in sentences:
        for i in range(len(sentence) - 1):
            for j in range(i + 1, len(sentence)):
                if graph.has_edge(sentence[i], sentence[j]):
                    graph[sentence[i]][sentence[j]]['weight'] += 1
                else:
                    graph.add_edge(sentence[i], sentence[j], weight=1)

    return graph


In [ ]:
def textrank(graph, num_iterations=50, d=0.85):
    scores = {node: 1.0 for node in graph.nodes}

    for _ in range(num_iterations):
        next_scores = {node: (1 - d) + d * sum(scores[neighbor] / graph.degree(neighbor) for neighbor in graph.neighbors(node)) for node in graph.nodes}
        scores = next_scores

    return scores

In [ ]:
import re
import spacy
import pandas as pd

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import pandas as pd
import re

def extract_sentences_with_medical_terms_and_questions(data, medical_terms, auxiliary_verbs):
    nlp = spacy.load("en_core_web_sm")
    relevant_sentences = []

    for text in data:
        doc = nlp(text)
        for sent in doc.sents:
            starts_with_wh_word = re.match(r"^[WwHh].*", sent.text)
            # Check if the first token is an auxiliary verb
            starts_with_auxiliary_verb = sent and sent[0].text.lower() in auxiliary_verbs and sent[0].dep_ == "aux"

            contains_medical_term = any(word in medical_terms for word in sent.text.split())
            ends_with_question_mark = sent.text.endswith('?')

            if starts_with_wh_word or starts_with_auxiliary_verb or contains_medical_term or ends_with_question_mark:
                relevant_sentences.append(sent.text)

    return relevant_sentences

# Read the CSV file
df = pd.read_excel("MeQSum.xlsx")

# Extract the first 5 rows from the "CHQ" and "Summary" columns
data_to_process = df.head(1001)

# Read medical terms from file
with open("medical_terms.txt", "r") as file:
    medical_terms = file.read().splitlines()

# Auxiliary verbs
auxiliary_verbs = ["am", "is", "are", "was", "were", "be", "been", "being",
                   "have", "has", "had", "do", "does", "did", "can", "could",
                   "shall", "should", "will", "would", "may", "might", "must"]

# Create a new DataFrame to store the results
new_df = pd.DataFrame(columns=["CHQ", "Summary", "Extracted Relevant Sentence"])

for index, row in data_to_process.iterrows():
    chq = row["CHQ"]
    summary = row["Summary"]
    relevant_sentences = extract_sentences_with_medical_terms_and_questions([chq], medical_terms, auxiliary_verbs)

    # Store the results in the new DataFrame
    new_df = new_df.append({"CHQ": chq, "Summary": summary, "Extracted Relevant Sentence": " ".join(relevant_sentences)}, ignore_index=True)

# Save the new DataFrame to a new CSV file
new_df.to_csv("fi.csv", index=False)


<ipython-input-14-a78e683dac8b>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({"CHQ": chq, "Summary": summary, "Extracted Relevant Sentence": " ".join(relevant_sentences)}, ignore_index=True)
<ipython-input-14-a78e683dac8b>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({"CHQ": chq, "Summary": summary, "Extracted Relevant Sentence": " ".join(relevant_sentences)}, ignore_index=True)
<ipython-input-14-a78e683dac8b>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({"CHQ": chq, "Summary": summary, "Extracted Relevant Sentence": " ".join(relevant_sentences)}, ignore_index=True)
<ipython-input-14-a78e683dac8b>:48: FutureWarning: The frame.append method i